# TD numérique n° 1 (MP) : tracé de lignes de champs



> **Capacité mise en oeuvre**  
> à l’aide d’un langage de programmation, tracer quelques lignes de champ électrostatique pour une distribution de charges donnée.

Le langage utilisé est la langage python.

## I - Principe

L’exercice consiste à faire tracer les surfaces équipotentielles et les lignes de champ électrique d’une distribution de charges ponctuelles.

En pratique, étant donnée une distribution de charges discrètes, on calcule le potentiel électrique $V$ total sur une surface rectangulaire, discrétisée à l’aide d’une grille $N_x \times N_y$ de points de coordonnées ($x_i$, $y_j$) régulièrement espacés :
$$x_i=i \times a \qquad ;\qquad y_i=j \times a\ ,$$
où le pas de la grille est pris égal à $a = 10^{-6}\mbox{ m}$. En 
chaque point de la grille, on calcule le potentiel 
$$ V_{i,j}=V(x_i, y_j)$$ 
en appliquant le théorème de superposition. Le potentiel créé par une charge $q = ne$ ($n \in \mathbb{Z}^*$) à une distance $r$ de la charge s’écrit :
$$V=\frac{n\tilde{e}}{r}\ ,$$
où $\tilde{e}$ est la charge réduite : 
$$\tilde{e} =\frac{e}{4\pi\epsilon_0}\ .$$
Le champ électrique est alors calculé à partir du gradient du potentiel. Les surfaces équipotentielles et les lignes de champs sont ensuite tracées à l’aide de fonctions dédiées de matplotlib.

## II - Constitution de la distribution de charge

> 1. Commencer par importer les bibliothèques numpy et matplotlib en exécutant la cellule suivante.

In [2]:
#%%  question 1 : importation

import numpy as np
import matplotlib.pyplot as plt


> 2. Initialiser les différents paramètres : les tailles $N_x$, $N_y$ (on commencera avec $N_x=N_y=200$) et le pas $a$ de la grille, ainsi que la charge réduite $\tilde e$. Exécuter la cellule.

In [3]:
#%%  question 2 : définitions et initialisations

Nx=200                                  # taille de la grille (Nx x Ny)
Ny=200
e=1.6e-19                               # charge élémentaire
epsilon0=8.85e-12                       # permittivité du vide
etilde=e/(4*np.pi*epsilon0)             # charge réduite
a=1e-6                                  # pas de la grille 

Chaque charge de la distribution est représentée par la liste [ q , i , j ], où :
* q est sa charge électrique ;
* i et j les indices (entiers) précisant sa position sur la grille.

La distribution de charge est alors constituée par la liste des différentes charges. Cette liste de listes est nommée *distrib* dans le programme. La fonction **saisie()** permet d’effectuer la saisie de la distribution au clavier.

> 3. Exécuter la cellule et saisir une distribution (saisir une charge nulle pour terminer). 

In [4]:
#%%  question 3 : saisie de la distribution de charge

def saisie():                           # saisie de la distribution de charge
    distribution=[]                          # initialisation de la distribution des charges
    nq=1                                # numérotation des charges 
    cq=input('Charge n°'+str(nq)+' : saisir un nombre entier de valeurs de e (0 pour terminer).')
    q=int(cq)*etilde                    # calcul de la charge 
    while q !=0 :
        ciq=input('Charge n°'+str(nq)+' : saisir son abscisse (entier compris entre 0 et '+str(Nx-1)+' ).')
        iq=int(ciq)
        cjq=input('Charge n°'+str(nq)+' : saisir son ordonnée (entier compris entre 0 et '+str(Ny-1)+' ).')
        jq=int(cjq)
        distribution.append([q,iq,jq])
        nq=nq+1
        cq=input('Charge n°'+str(nq)+' : saisir un nombre entier de valeurs de e (0 pour terminer).')
        q=int(cq)*etilde
    return distribution

distrib=saisie()                        # constitution de la distribution des charges


> 4. Compléter la fonction **visu** qui prend *distrib* en paramètres et affiche la distribution. On utilisera la fonction **plt.imshow** qui affiche un tableau numpy à deux dimensions comme une photo. Dans ce tableau, la présence d’une charge en point (i,j) sera traduite par la valeur 1, son absence par la valeur 0. Exécuter la cellule.

In [5]:
#%%  question 4. : visualisation de la distribution

def visu(distrib):
    présence=np.zeros((Nx,Ny))         # initialisation du tableau
    for charge in distrib :
        i,j=int(charge[1]),int(charge[2])
        présence[i,j]=1                # présence d'une charge
    plt.imshow(présence)
      
plt.clf()
visu(distrib)   # affichage de la distribution
plt.show()

## III - Tracé des surfaces équipotentielles

On souhaite calculer le potentiel $V$ en chaque point de la grille et stocker les valeurs dans un tableau numpy *V* à deux dimensions. Pour cela, on introduit 
* une fonction **distance** qui prend comme paramètres les caractéristiques d’une charge (sous forme d'une liste), les coordonnées i et j d’un point M sur la grille et renvoie la distance de la charge au point M ;
* une fonction **potentiel** qui prend comme paramètres les caractéristiques d’une charge, les coordonnées i et j 
du point M et renvoie la valeur du potentiel en M ;
* une fonction **calcul_potentiel** qui prend comme paramètres distrib et renvoie le tableau *V* des valeurs de potentiels en tout point de la grille.

Remarquer qu’en chaque point de la grille où est placée une charge, le potentiel diverge. Une méthode propre consiste à exclure ces points du calculs du potentiel, puis du champ électrique. On propose ici de rendre
finie la valeur du potentiel en remplaçant par a/10 la valeur 0 renvoyée
par la fonction distance.

> 5. Compléter les fonction **distance**, **potentiel** et **calcul_potentiel**, puis exécuter la cellule.

In [6]:
#%%  question 5 : calcul du potentiel

def distance(charge,i,j):
    icharge=charge[1]
    jcharge=charge[2]
    d_pix=((i-icharge)**2+(j-jcharge)**2)**(1/2)
    return d_pix*a

def potentiel(charge,i,j):
    ri=distance(charge,i,j)
    if ri==0 :
        return charge[0]/(a/10)
    else :
        return charge[0]/ri

def calcul_potentiel(distrib):
    V=np.zeros((Nx,Ny))  
    for i in range(Nx):
        for j in range(Ny):
            Vij=0
            for charge in distrib :
                Vij+=potentiel(charge,i,j)
            V[i,j]=Vij
    return V

V=calcul_potentiel(distrib)


Les lignes équipotentielles sont tracées en utilisant la fonction **plt.contour**, qui prend comme argument le tableau *V* de valeurs de potentiel et le tableau *Valeurs* qui précise pour quelles valeurs de V sont tracées les équipotentielles. Le tableau *Valeurs* pourra être constitué avec la fonction **np.linspace (a,b,num=n)** de numpy, 
qui renvoie un tableau contenant $n$ valeurs régulièrement espacées d'un l'intervalle $[a,b]$.

> 6. Compléter la première ligne, puis exécuter la cellule. 

In [9]:
#%%  question 6 : tracé des lignes équipotentielles

Valeurs=np.linspace(np.min(V)/80,np.max(V)/80,num=101)
plt.clf()
plt.imshow(V)
plt.contour(V,Valeurs,cmap='hot')
plt.show()


## IV - Tracé des lignes de champ électrostatique

Le champ électrique s’écrit $\vec E =-\overrightarrow{\mbox{grad}}[V]$, soit
$$E_x = -\frac{\partial V}{\partial x} \qquad\mbox{et}\qquad  E_y = -\frac{\partial V}{\partial y}\ .$$
La discrétisation de ces relations se traduit par
$$E_x(i,j) = \frac{V(i+1,j)-V(i,j)}{a} \qquad\mbox{et}\qquad E_y(i,j) = \frac{V(i,j+1)-V(i,j)}{a}\ .$$
On souhaite calculer les composantes du champ électrique en chaque point de la grille (excepté sur la
dernière ligne et la dernière colonne) et stocker les valeurs dans deux tableaux numpy *Ex* et *Ey* à 
deux dimensions. Pour cela, on introduit :
* une fonction **champE** qui prend comme paramètres le tableau *V* des valeurs du potentiel, les coordonnées i et j d’un point M sur la grille et renvoie les valeurs des composantes du champ au point M ;
* une fonction **calcul_champE** qui prend comme paramètres le tableau *V* et renvoie les tableaux de valeurs des composantes du champ électrique en tout point de la grille, privée de la dernière ligne et de la dernière colonne.

> 7. Compléter les fonctions **champE** et **calcul_champE** et exécuter la cellule.

In [10]:
#%%  Cellule 7 : calcul du champ électrique

def champE(V,i,j):
    Ex=(V[i,j]-V[i,j+1])/a
    Ey=(V[i,j]-V[i+1,j])/a
    return Ex,Ey

def calcul_champE(V):
    Ex=np.zeros((Nx,Ny))
    Ey=np.zeros((Nx,Ny))
    for i in range(Nx-1):
        for j in range(Ny-1):
            Ex[i,j],Ey[i,j]=champE(V,i,j)
    return Ex,Ey

Ex,Ey=calcul_champE(V)


> 8. Aux lignes équipotentielles déjà tracées, superposer les lignes de champs en utilisant la fonction **plt.streamplot**, qui prend comme argument, dans cet ordre :
    * un tableau comportant les différentes valeurs prises par j ;
    * un tableau comportant les différentes valeurs prises par i ;
    * les tableaux Ex et Ey.

In [11]:
#%%  Cellule 8 : tracé des lignes équipotentielles et des lignes de champs

import matplotlib
matplotlib.rcParams['figure.figsize']=[9,9]

plt.clf()
plt.imshow(V)
plt.contour(V,Valeurs,cmap='hot')
plt.streamplot(np.linspace(0,Ny-1,Ny), np.linspace(0,Nx-1,Nx), Ex, Ey, color="red", linewidth=1, density=1)  
plt.show()


> 9. Vérifier l’allure des lignes de champs et des surfaces équipotentielles dans les cas simples (une charge), 
puis dans les situations décrites dans l’exercice n°6 du TD n° 6.